# Información previa



## datos
faltan en pollo 1010130 y 1010387


## Diseño y esquema codigo

## Apuntes Cod 

```{python}
Data['Peso neto KG']=Data['Peso neto'].str.split('KG', expand=True)[[0]]
Data.pop('Peso neto')
Data['Cantidad Unidades CJ']=Data['Cantidad Unidades'].str.split('CJ', expand=True)[[0]]
Data.pop('Cantidad Unidades')
Data['Precio Unitario CLP / KG']=Data['Precio Unitario'].str.split('CLP / KG', expand=True)[[0]]



    if(x[len(x)-3:]=='USD'):
        x=x.split(' USD')[1]
    elif(x[len(x)-3:]=='CLP'):
        x=x.split(' CLP')[1]


```

## Este es un extracto del paper:


 

	
Ventas pasadas con retraso  k, es decir, ventas diarias para el SKU considerado de los últimos k días (tw k : ventana de tiempo de tamaño k ).

	
* Variables binarias para caracterizar días especiales, como:
    * Pago , que caracteriza los días al final de cada mes cuando las personas reciben el pago mensual.
    * Pago intermedio , que caracteriza los días al final de cada quincena en que las personas reciben dos pagos semanales.
    * Antes de las vacaciones , caracterizando los días antes de las vacaciones.
    * Vacaciones , que caracterizan las vacaciones.
    * Independencia , que caracteriza los días de la independencia de Chile (18 y 19 de septiembre).
    * Santa , caracterizando los días de la semana antes del este.
    * Vacaciones , caracterizando los días que pertenecen al período de vacaciones de verano (enero y febrero).
    * Verano , que caracteriza los días de verano (1 de octubre-31 de marzo).	
    * Año nuevo (1 de enero), que caracteriza el único día de cierre de supermercados en Chile.
	
	
* Variables de precio:
    * Precio A  =  precio original del SKU considerado en Economax.
    * Precio B  =  (Precio A) / (Precio máximo en micro-mercado).
    * Precio C  =  (Precio A) / (Precio mínimo en micro-mercado).

		
Observación : Las variables “Precio máximo en micro-mercado” y “Precio mínimo en micro-mercado” son el precio máximo (mínimo) para un determinado SKU en el micro-mercado relevante y son proporcionados por una empresa de investigación de mercado.
		
	

	
 

# Package Install & Open Data


## Import

### import general 

In [1]:
# =============================================================================
# Importar paquetes y abrir Data
# =============================================================================

#gestion DF
import pandas as pd

#Funciones mat y matricicales
import numpy as np

#ploteo simple
import matplotlib.pyplot as plt

# Graficas bonitas
from plotnine import *

#para ajustar las fechas
import datetime

#para calcular el skew de un array() o list()
from scipy.stats import skew 

#mode function
import statistics 

from mizani.breaks import date_breaks
from mizani.formatters import date_format

In [2]:
from time import time

### import and function 

In [3]:

import warnings
import pandas as pd
import matplotlib.pyplot as plt
# prophet model 
from fbprophet import Prophet
# prophet preformance
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

# don't do this 
warnings.filterwarnings('ignore')
# "high resolution"
%config InlineBackend.figure_format = 'retina'

Importing plotly failed. Interactive plots will not work.


##  Data Facturación

Link de descarga de BW

[08_06_21_Fact_CL_Mater_Canales_10_06_21](https://gestion.agrosuper.cl/irj/servlet/prt/portal/prtroot/pcd!3aportal_content!2fcom.sap.pct!2fplatform_add_ons!2fcom.sap.ip.bi!2fiViews!2fcom.sap.ip.bi.bex?BOOKMARK=4V7XT6V546W73QKC7BVTLKF4Y)

### Selección de la zona de estudio 

In [4]:
#Nombre del producto
Fecha='08_06_21'

# =============================================================================
# Ajuste de los datos
# =============================================================================
File_name='Fact_CL_Mater_Canales_'+Fecha
col_names=pd.read_csv("Data/"+File_name+".csv",
                    encoding="utf-8",sep=";",nrows=0).columns
types_dict = {col: str for col in col_names}

Data = pd.read_csv("Data/"+File_name+".csv",
                   encoding="utf-8",sep=";",dtype=types_dict)
Data.head(1)

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio
0,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,02.01.2019,MI,36,81.788 CLP,2.250 CLP


### Ajuste facturaciones

In [5]:
# =============================================================================
# Ajustar la data
# =============================================================================
def Ajuste_data(data):
    df=data.copy()
    # Funciones para eliminar los putos y cambiar las , por .
    def categ_uniq(x):
        return x[len(x)-3:]

    df['Unidad Venta']=df['Venta Neta'].apply(categ_uniq)
    df['Unidad Precio']=df['Precio Promedio'].apply(categ_uniq)
    
    def CLP_Dolar(x):
        x=str(x)
        x=x.replace('.', '')
        x=x.replace(',', '.')
        if(x[len(x)-3:]=='USD'):
            x=x.split(' USD')[0]
            x=float(x)
            #x=x*720
        elif(x[len(x)-3:]=='CLP'):
            x=x.split(' CLP')[0]
            x=float(x)
        return x
    
    def change_num_point(x):
        x=str(x)
        return x.replace('.', '')
    def change_num_coma(x):
        x=str(x)
        return x.replace(',', '.')
    def delete_asterisco(x):
        x=str(x)
        return x.replace('*','0' )
    
    #proceso iterativo, 
    for i in ['Kilos Venta KG','Venta Neta','Precio Promedio']:
    # se elimina el punto definido para los miles
        if(i!='Kilos Venta KG'):
            df[i]=df[i].apply(CLP_Dolar)
        else:
            df[i]=df[i].apply(change_num_point)
            # se reemplaza la coma por un punto, para definir los decimales correctamente
            df[i]=df[i].apply(change_num_coma)
        # se elimina el valor * 
        df[i]=df[i].apply(delete_asterisco)
        df[i]=df[i].astype('float64')
        # Se pasa de ser un str a un int
        print(i)
        #df[i]=df[i].astype('int64')

    #Separacion fecha
    df['Semana']=df['Año natural/Semana'].str.split('.', expand=True)[[0]]
    df['Año']=df['Año natural/Semana'].str.split('.', expand=True)[[1]]

    df['Semana']=df['Semana'].astype('int64')
    df['Año']=df['Año'].astype('int64')
    
    #
    df["Día natural"] = df["Día natural"].apply(lambda x: 
                                                datetime.datetime.strptime(x,
                                                                           '%d.%m.%Y').date())
    df['Año']=df['Año'].astype('int64')
    return df

In [22]:
data1=Ajuste_data(Data)
data1.head(3)
#data=<data[data['Año']>2018]

Kilos Venta KG
Venta Neta
Precio Promedio


,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
0,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-02,MI,36.0,81788.0,2250.0,CLP,CLP,1,2019
1,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-03,JU,333.0,583204.0,1750.0,CLP,CLP,1,2019
2,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-05,SA,219.0,397031.0,1815.0,CLP,CLP,1,2019


##  Data Facturación Concatena a la anterior
Link de descarga de BW
[08_06_21_Fact_CL_Mater_Canales_10_06_21](https://gestion.agrosuper.cl/irj/servlet/prt/portal/prtroot/pcd!3aportal_content!2fcom.sap.pct!2fplatform_add_ons!2fcom.sap.ip.bi!2fiViews!2fcom.sap.ip.bi.bex?BOOKMARK=4V7XT6V546W73QKC7BVTLKF4Y)

Contiene las facturaciones siguientes de todo chile para el listado de productos y para los canales de venta:
Foodservices; Canal Tradicional; Industriales; GRandes Clientes

### Selección de la zona de estudio 

In [7]:
#Nombre del producto
Fecha='08_06_21'
Fecha_fin='12_06_21'
# =============================================================================
# Ajuste de los datos
# =============================================================================
File_name=Fecha+'_Fact_CL_Mater_Canales_'+Fecha_fin
col_names=pd.read_csv("Data/"+File_name+".csv",
                    encoding="utf-8",sep=";",nrows=0).columns
types_dict = {col: str for col in col_names}

Data2 = pd.read_csv("Data/"+File_name+".csv",
                   encoding="utf-8",sep=";",dtype=types_dict)
Data2.head(1)

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio
0,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,6,23.2021,08.06.2021,MA,273,697.541 CLP,2.555 CLP


In [8]:
data2=Ajuste_data(Data2)

Kilos Venta KG
Venta Neta
Precio Promedio


## Unificar 

### Primera y ultima fecha 

In [9]:
data2['Día natural'].max()

datetime.date(2021, 6, 11)

In [10]:
data2['Día natural'].min()

datetime.date(2021, 6, 8)

In [11]:
data1['Día natural'].max()

datetime.date(2021, 6, 7)

In [12]:
data1['Día natural'].min()

datetime.date(2019, 1, 2)

### Concatenar 

In [13]:
data=pd.concat([data1, data2])
data.describe(include='all')

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
count,449430,449430,449430,449430,449430,449430,449430,449430,449430,449430,449430,449430.000000,4.494300e+05,449430.000000,449430,449430,449430.000000,449430.000000
unique,2,13,4,6,6,34,34,12,128,788,7,NaN,NaN,NaN,3,3,NaN,NaN
top,Pollo,1010078,Tradicional,Z003,Centro Sur,Concepcion,T104,3,49.2019,2019-04-18,MI,NaN,NaN,NaN,CLP,CLP,NaN,NaN
freq,238796,52145,210463,124471,124471,22980,22980,49744,4061,731,77977,NaN,NaN,NaN,449015,449015,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607.307383,9.601389e+05,1849.338858,NaN,NaN,23.717983,2019.762070
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1245.086148,1.802542e+06,724.569747,NaN,NaN,15.067890,0.732747
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000e+00,0.000000,NaN,NaN,1.000000,2019.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.000000,1.205830e+05,1307.000000,NaN,NaN,11.000000,2019.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.000000,3.469035e+05,1686.000000,NaN,NaN,22.000000,2020.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,555.000000,9.725628e+05,2278.000000,NaN,NaN,36.000000,2020.000000


In [14]:
data['Día natural'].min()

datetime.date(2019, 1, 2)

In [15]:
data['Día natural'].max()

datetime.date(2021, 6, 11)

# Predicción  Prophet


In [16]:
y_col='Kilos Venta KG'
y_value='Volumen de ventas en KG'
y_title= 'Demanda'+'del canal tradicion'

## Descripción 

In [17]:
data.head()

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
0,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-02,MI,36.0,81788.0,2250.0,CLP,CLP,1,2019
1,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-03,JU,333.0,583204.0,1750.0,CLP,CLP,1,2019
2,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,01.2019,2019-01-05,SA,219.0,397031.0,1815.0,CLP,CLP,1,2019
3,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,02.2019,2019-01-07,LU,128.0,258591.0,2017.0,CLP,CLP,2,2019
4,Cerdo,1020253,Tradicional,Z001,Santiago,Huechuraba,T101,1,02.2019,2019-01-08,MA,95.0,210790.0,2223.0,CLP,CLP,2,2019


In [18]:
data.columns

Index(['Sector', 'Material', 'Tipo de cliente', 'Zona de ventas',
       'Zona de ventas.1', 'Oficina de Ventas Unifica',
       'Oficina de Ventas Unifica.1', 'Mes', 'Año natural/Semana',
       'Día natural', 'Día de la semana', 'Kilos Venta KG', 'Venta Neta',
       'Precio Promedio', 'Unidad Venta', 'Unidad Precio', 'Semana', 'Año'],
      dtype='object')

In [19]:
print(data['Tipo de cliente'].unique())
print(data['Zona de ventas.1'].unique())
print(data['Oficina de Ventas Unifica'].unique())
print(data['Material'].unique())

['Tradicional' 'Industriales' 'Grandes Clientes' 'Foodservice']
['Santiago' 'Centro Norte' 'Centro Sur' 'Sur' 'Norte' 'Venta Directa']
['Huechuraba' 'Lo Espejo' 'Quilin' 'Sucursal Miraflores' 'Coquimbo'
 'Hijuelas' 'Ovalle' 'San Antonio' 'San Felipe' 'Viña del Mar' 'Chillan'
 'Concepcion' 'Curico' 'Los Angeles' 'Rancagua' 'Talca' 'Castro'
 'Coyhaique' 'Osorno' 'Puerto Montt' 'Punta Arenas' 'Temuco' 'Valdivia'
 'Antofagasta' 'Arica' 'Calama' 'Copiapo' 'Iquique' 'Vallenar'
 'VD FoodService' 'VD Industrial' 'VD Grandes Clientes'
 'Sucursal Dieciocho' 'Vendedores San Pablo']
['1020253' '1020042' '1020027' '1020013' '1020010' '1020003' '1020001'
 '1010695' '1010267' '1010078' '1010077' '1010048' '1010002']


## función 

In [20]:
def Df_ubic(data_X,col,Str):
    df=data_X.copy()
    df=df[df[col]==Str]
    return df

In [21]:
def Predict_sem(D_Train,periodos=5):
    D_Train.columns = ['ds','y']
    
    # convert dates from string to datetime
    D_Train.ds = pd.to_datetime(D_Train.ds)
    #print(D_Train.dtypes)
    
    # set prophet model 
    prophet = Prophet(changepoint_prior_scale=0.15, yearly_seasonality=True)
    
    # fit $wmt data to model
    prophet.fit(D_Train)
    
    
    
    # build future dataframe for 5 years
    build_forecast = prophet.make_future_dataframe(periods= periodos , freq='D')
    #prophet.make_future_dataframe(periods= periodos, freq='D')
    
    # forecast future df w/ model
    forecast = prophet.predict(build_forecast)
    
    return(forecast)

## Predicción de todo chile

In [24]:
#inicializa con un Data Frame vacio
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat',
                            'Material'])

df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

#Crear un listado de los materiales disponibles en la data
Listado_de_Materiales=data['Material'].unique()

for m in Listado_de_Materiales:
    #m=Listado_de_Materiales[0]
    dfm=Df_ubic(data,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]
    
    #Realizar predicción
    dfmPred=Predict_sem(dfm,8)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(8)
    dfmPred.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)
    dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
    
    #Sector del producto(Tipo de carne)
    Sec=Df_ubic(data,'Material',m).Sector.unique()[0]
    dfmPred['Sector']=[Sec for i in range(dfmPred.shape[0])]
    
    dfmPred=dfmPred.reset_index()
    dfmPred.pop('index')
    df2=pd.concat([df2, dfmPred])

    
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

,Día natural,Predicción Inferior,Predicción Superior,Predicción central,Material,Sector
0,2021-06-12,9280.061862,18009.151910,13680.623828,1020253,Cerdo
1,2021-06-13,2178.539040,11054.096855,6663.595223,1020253,Cerdo
2,2021-06-14,6237.783072,15022.564370,10707.495410,1020253,Cerdo
3,2021-06-15,8055.888034,17049.803913,12744.569866,1020253,Cerdo
4,2021-06-16,9488.779429,18681.788006,14287.205309,1020253,Cerdo


### CSV SAVE

In [25]:

min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

nan
nan


In [24]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df2.to_csv('Predicción Todo Chile del'+min_time+' al '+max_time+'.csv', index=False)

### Analisis Exploratorio

In [25]:
df2.groupby(['Material','Sector']).sum()

,,Predicción Inferior,Predicción Superior,Predicción central
Material,Sector,,,
1010002,Pollo,84818.162271,2.607600e+05,173854.359913
1010048,Pollo,33290.964800,3.602771e+05,196939.927339
1010077,Pollo,11650.815669,5.984597e+04,36042.905637
1010078,Pollo,668575.656901,1.212335e+06,939917.733762
1010267,Pollo,149571.619144,3.881611e+05,267047.687966
1010695,Pollo,7389.255280,1.560197e+05,82107.662351
1020001,Cerdo,57496.199370,1.360212e+05,96515.554877
1020003,Cerdo,3896.554147,8.699460e+04,44989.481031
1020010,Cerdo,160097.767415,2.906070e+05,226217.445300


## Predicción POR CANAL EN todo chile

In [26]:
data.columns

Index(['Sector', 'Material', 'Tipo de cliente', 'Zona de ventas',
       'Zona de ventas.1', 'Oficina de Ventas Unifica',
       'Oficina de Ventas Unifica.1', 'Mes', 'Año natural/Semana',
       'Día natural', 'Día de la semana', 'Kilos Venta KG', 'Venta Neta',
       'Precio Promedio', 'Unidad Venta', 'Unidad Precio', 'Semana', 'Año'],
      dtype='object')

In [27]:
#inicializa con un Data Frame vacio
df3 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat',
                            'Material'])

Listado_de_Canales=data['Tipo de cliente'].unique()

for c in Listado_de_Canales:
    #m=Listado_de_Materiales[0]
    dfc=Df_ubic(data,'Tipo de cliente',c)

    #Crear un listado de los materiales disponibles en la data
    Listado_de_Materiales=dfc[dfc.Año==2021].Material.unique()     


    for m in Listado_de_Materiales:
        #m=Listado_de_Materiales[0]
        dfm=Df_ubic(dfc,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

        #Realizar predicción
        dfmPred=Predict_sem(dfm,8)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(8)

        dfmPred['Material']=[m for i in range(dfmPred.shape[0])]

        #Sector del producto(Tipo de carne)
        Sec=Df_ubic(data,'Material',m).Sector.unique()[0]
        dfmPred['Sector']=[Sec for i in range(dfmPred.shape[0])]
        
        #Canal de ventas
        dfmPred['Tipo de cliente']=[c for i in range(dfmPred.shape[0])]
        
        dfmPred=dfmPred.reset_index()
        dfmPred.pop('index')
        df3=pd.concat([df3, dfmPred])

    
df3.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

,ds,yhat_lower,yhat_upper,yhat,Material,Sector,Tipo de cliente
0,2021-06-12,6406.265109,13533.990484,9947.250821,1020253,Cerdo,Tradicional
1,2021-06-13,24.276411,7373.453503,3799.199531,1020253,Cerdo,Tradicional
2,2021-06-14,3677.006285,10776.902322,7040.434877,1020253,Cerdo,Tradicional
3,2021-06-15,4470.370458,11689.055196,8256.586063,1020253,Cerdo,Tradicional
4,2021-06-16,5976.493961,13600.180251,9845.337275,1020253,Cerdo,Tradicional


In [28]:
df3.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

In [29]:
df3.tail()

,Día natural,Predicción Inferior,Predicción Superior,Predicción central,Material,Sector,Tipo de cliente
3,2021-06-15,-114.763627,2723.506083,1273.466913,1010002,Pollo,Foodservice
4,2021-06-16,105.975954,3010.555070,1570.806590,1010002,Pollo,Foodservice
5,2021-06-17,261.730742,3166.752202,1664.442530,1010002,Pollo,Foodservice
6,2021-06-18,109.894375,3020.018052,1609.255150,1010002,Pollo,Foodservice
7,2021-06-19,-525.985296,2493.111918,1043.100181,1010002,Pollo,Foodservice


### CSV SAVE

In [30]:
min_time=df3['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df3['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-06-10
2021-06-19


In [31]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df3.to_csv('Predicción Todo Chile del '+min_time+' al '+max_time+' Diferenciando por canal.csv', index=False)

### Exploracion 

In [32]:
D_sem=df3.groupby(['Sector','Material','Tipo de cliente']).sum().reset_index()
D_sem

,Sector,Material,Tipo de cliente,Predicción Inferior,Predicción Superior,Predicción central
0,Cerdo,1020001,Foodservice,-888.759173,2.160379e+03,641.482445
1,Cerdo,1020001,Grandes Clientes,3740.907517,2.122964e+04,12313.957349
2,Cerdo,1020001,Industriales,-3228.427499,4.298642e+03,598.826237
3,Cerdo,1020001,Tradicional,51148.957092,1.154099e+05,82984.604153
4,Cerdo,1020003,Foodservice,925.403909,6.722116e+03,3848.203218
5,Cerdo,1020003,Grandes Clientes,-8538.051060,1.244389e+04,2024.732035
6,Cerdo,1020003,Industriales,-2562.101873,4.556648e+03,1030.964945
7,Cerdo,1020003,Tradicional,5522.046019,6.916304e+04,37314.907407
8,Cerdo,1020010,Foodservice,4863.283743,3.226113e+04,18736.320399
9,Cerdo,1020010,Grandes Clientes,13411.384928,5.101877e+04,32359.292393


In [33]:
D_sem[D_sem.Material=='1010130']

,Sector,Material,Tipo de cliente,Predicción Inferior,Predicción Superior,Predicción central


In [34]:
df3.groupby(['Sector','Material','Tipo de cliente']).sum()

Predicción Inferior  Predicción Superior  \
Sector Material Tipo de cliente                                              
Cerdo  1020001  Foodservice               -888.759173         2.160379e+03   
                Grandes Clientes          3740.907517         2.122964e+04   
                Industriales             -3228.427499         4.298642e+03   
                Tradicional              51148.957092         1.154099e+05   
       1020003  Foodservice                925.403909         6.722116e+03   
                Grandes Clientes         -8538.051060         1.244389e+04   
                Industriales             -2562.101873         4.556648e+03   
                Tradicional               5522.046019         6.916304e+04   
       1020010  Foodservice               4863.283743         3.226113e+04   
                Grandes Clientes         13411.384928         5.101877e+04   
                Industriales             -7267.684571         1.158543e+04   
                Tradicional             120438.283129         2.139796e+05   
       1020013  Foodservice               -357.676773         8.277412e+03   
                Grandes Clientes         17209.462125         1.051074e+05   
                Industriales            -11009.599319         1.897682e+04   
                Tradicional             163721.624116         3.482101e+05   
       1020027  Foodservice               -603.535857         4.778869e+03   
                Grandes Clientes         11313.058928         3.583842e+04   
                Industriales             -3192.367745         9.212470e+03   
                Tradicional              64515.702713         1.438172e+05   
       1020042  Foodservice               -544.733486         1.391377e+03   
                Grandes Clientes         -1791.379895         1.199725e+04   
                Industriales              3578.179861         1.456369e+04   
                Tradicional               2582.030050         2.955854e+04   
       1020253  Foodservice              13792.283931         2.648695e+04   
                Grandes Clientes            41.411013         1.618232e+04   
                Industriales              -679.920693         7.462508e+03   
                Tradicional              42438.710800         1.000663e+05   
Pollo  1010002  Foodservice              -2918.938484         2.041663e+04   
                Grandes Clientes          1337.983578         5.064655e+04   
                Industriales             -1123.967023         3.142174e+03   
                Tradicional              78527.429862         2.129739e+05   
       1010048  Foodservice              15673.237399         5.033347e+04   
                Grandes Clientes        -50681.381856         6.060784e+04   
                Industriales              -513.055484         4.975085e+03   
                Tradicional              39031.171154         2.711742e+05   
       1010077  Foodservice               -336.157972         4.323153e+03   
                Grandes Clientes         -5129.385901         3.414426e+03   
                Industriales             -1081.723833         8.426371e+02   
                Tradicional              12199.619928         5.540435e+04   
       1010078  Foodservice              45711.291759         8.111534e+04   
                Grandes Clientes          6899.814547         1.545496e+05   
                Industriales              1180.599203         1.973219e+04   
                Tradicional             575522.204381         1.005348e+06   
       1010267  Foodservice             104539.529501         2.103694e+05   
                Grandes Clientes        -29423.576120         3.044050e+04   
                Industriales            -29445.338312         2.448152e+04   
                Tradicional              37787.928040         1.740935e+05   
       1010695  Foodservice               4714.139560         1.783329e+04   
                Grandes Clientes        -21493.644801   

In [35]:
df3.groupby(['Sector','Material','Tipo de cliente']).sum()

Predicción Inferior  Predicción Superior  \
Sector Material Tipo de cliente                                              
Cerdo  1020001  Foodservice               -888.759173         2.160379e+03   
                Grandes Clientes          3740.907517         2.122964e+04   
                Industriales             -3228.427499         4.298642e+03   
                Tradicional              51148.957092         1.154099e+05   
       1020003  Foodservice                925.403909         6.722116e+03   
                Grandes Clientes         -8538.051060         1.244389e+04   
                Industriales             -2562.101873         4.556648e+03   
                Tradicional               5522.046019         6.916304e+04   
       1020010  Foodservice               4863.283743         3.226113e+04   
                Grandes Clientes         13411.384928         5.101877e+04   
                Industriales             -7267.684571         1.158543e+04   
                Tradicional             120438.283129         2.139796e+05   
       1020013  Foodservice               -357.676773         8.277412e+03   
                Grandes Clientes         17209.462125         1.051074e+05   
                Industriales            -11009.599319         1.897682e+04   
                Tradicional             163721.624116         3.482101e+05   
       1020027  Foodservice               -603.535857         4.778869e+03   
                Grandes Clientes         11313.058928         3.583842e+04   
                Industriales             -3192.367745         9.212470e+03   
                Tradicional              64515.702713         1.438172e+05   
       1020042  Foodservice               -544.733486         1.391377e+03   
                Grandes Clientes         -1791.379895         1.199725e+04   
                Industriales              3578.179861         1.456369e+04   
                Tradicional               2582.030050         2.955854e+04   
       1020253  Foodservice              13792.283931         2.648695e+04   
                Grandes Clientes            41.411013         1.618232e+04   
                Industriales              -679.920693         7.462508e+03   
                Tradicional              42438.710800         1.000663e+05   
Pollo  1010002  Foodservice              -2918.938484         2.041663e+04   
                Grandes Clientes          1337.983578         5.064655e+04   
                Industriales             -1123.967023         3.142174e+03   
                Tradicional              78527.429862         2.129739e+05   
       1010048  Foodservice              15673.237399         5.033347e+04   
                Grandes Clientes        -50681.381856         6.060784e+04   
                Industriales              -513.055484         4.975085e+03   
                Tradicional              39031.171154         2.711742e+05   
       1010077  Foodservice               -336.157972         4.323153e+03   
                Grandes Clientes         -5129.385901         3.414426e+03   
                Industriales             -1081.723833         8.426371e+02   
                Tradicional              12199.619928         5.540435e+04   
       1010078  Foodservice              45711.291759         8.111534e+04   
                Grandes Clientes          6899.814547         1.545496e+05   
                Industriales              1180.599203         1.973219e+04   
                Tradicional             575522.204381         1.005348e+06   
       1010267  Foodservice             104539.529501         2.103694e+05   
                Grandes Clientes        -29423.576120         3.044050e+04   
                Industriales            -29445.338312         2.448152e+04   
                Tradicional              37787.928040         1.740935e+05   
       1010695  Foodservice               4714.139560         1.783329e+04   
                Grandes Clientes        -21493.644801   

## Predicción selecionada 

### Sucursal 

In [36]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])


Listado_de_zonas=data['Zona de ventas.1'].unique()

#for z in Listado_de_zonas:
Z='Norte'

dfz=data#Df_ubic(data,'Zona de ventas.1',Z)

Listado_de_Sucursales=dfz['Oficina de Ventas Unifica'].unique()

#for y in Listado_de_Sucursales:

y='Arica'
dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


Listado_de_Materiales=dfy['Material'].unique()

#for m in Listado_de_Materiales:
m=Listado_de_Materiales[0]
dfm=Df_ubic(dfy,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(6)
dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
dfmPred=dfmPred.reset_index()
dfmPred.pop('index')
df2=pd.concat([df2, dfmPred])
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-13,-57.465248,135.318944,34.900493,1020253,Arica,Norte
1,2021-06-14,-48.317796,150.183209,49.715464,1020253,Arica,Norte
2,2021-06-15,-64.998028,141.065270,38.686661,1020253,Arica,Norte
3,2021-06-16,-24.261868,179.338739,78.271628,1020253,Arica,Norte
4,2021-06-17,-45.382790,160.775120,57.973342,1020253,Arica,Norte


### Zona 

In [37]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])


Listado_de_zonas=data['Zona de ventas.1'].unique()

#for z in Listado_de_zonas:
Z='Santiago'

dfz=data#Df_ubic(data,'Zona de ventas.1',Z)


Listado_de_Materiales=dfz['Material'].unique()

#for m in Listado_de_Materiales:
m=Listado_de_Materiales[0]
dfm=Df_ubic(dfz,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

dfmPred=Predict_sem(dfm,7)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(6)
dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
#dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
dfmPred=dfmPred.reset_index()
dfmPred.pop('index')
df2=pd.concat([df2, dfmPred])
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-13,2255.909436,11048.439429,6663.595223,1020253,NaN,Santiago
1,2021-06-14,6155.265265,14977.040561,10707.495410,1020253,NaN,Santiago
2,2021-06-15,8321.529271,17012.659354,12744.569866,1020253,NaN,Santiago
3,2021-06-16,9872.628696,18703.024312,14287.205309,1020253,NaN,Santiago
4,2021-06-17,10764.097397,19676.243419,15291.635632,1020253,NaN,Santiago


# Predicción Iterativa  

## Iteración 1 zona

In [25]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])



Listado_de_zonas=data['Zona de ventas.1'].unique

start_time = time()



#for Z in Listado_de_zonas:
Z='Norte'

dfz=Df_ubic(data,'Zona de ventas.1',Z)

Listado_de_Sucursales=dfz[dfz['Día natural'] >datetime.date(2021, 6, 2)]['Oficina de Ventas Unifica'].unique()

for y in Listado_de_Sucursales:
    #y='Huechuraba'
    dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


    Listado_de_Materiales=dfy['Material'].unique()

    for m in Listado_de_Materiales:
        #m=Listado_de_Materiales[0]
        material_time=time()
        print(y,m)


        dfm=Df_ubic(dfy,'Material',m).groupby('Día natural').sum().reset_index()[['Día natural',y_col]]

        dfmPred=Predict_sem(dfm,8)[['ds', 'yhat_lower', 'yhat_upper', 'yhat']].tail(8)
        dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
        dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
        dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
        dfmPred=dfmPred.reset_index()
        dfmPred.pop('index')
        df2=pd.concat([df2, dfmPred])
        
        print('El producto demoró',(time() - material_time),Z,y)
        print('tiempo que lleva corriendo',(time() - start_time)/60,'Min')
    
print('tiempo bloque completo',(time() - start_time)/60,'Min')
df2.head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Antofagasta 1020253


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 0.


El producto demoró 1.5032901763916016 Norte Antofagasta
tiempo que lleva corriendo 0.028024065494537353 Min
Antofagasta 1020042


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.6749370098114014 Norte Copiapo
tiempo que lleva corriendo 1.7515113870302836 Min
Copiapo 1010048


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.8811137676239014 Norte Copiapo
tiempo que lleva corriendo 1.7828716238339741 Min
Copiapo 1010002
El producto demoró 1.7154719829559326 Norte Copiapo
tiempo que lleva corriendo 1.8114878416061402 Min
tiempo bloque completo 1.8114961862564087 Min


,ds,yhat_lower,yhat_upper,yhat,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-12,25.312015,541.001204,276.389096,1020253,Antofagasta,Norte
1,2021-06-13,-179.033391,333.804823,74.823879,1020253,Antofagasta,Norte
2,2021-06-14,-154.935995,374.118050,101.708623,1020253,Antofagasta,Norte
3,2021-06-15,-173.416171,354.824576,98.074634,1020253,Antofagasta,Norte
4,2021-06-16,-151.460983,399.836341,123.959807,1020253,Antofagasta,Norte


In [26]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

In [38]:
165.4798605442047/60

2.7579976757367453

### CSV SAVE 

In [39]:

min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2019-06-19
2021-06-19


In [40]:
#Error_DF_Sum.to_csv('E_interv_Dem.csv', index=False)
df2.to_csv('Predicción Zona '+Z+max_time+'.csv', index=False)

## Iteración completa Zona, Sucursal

In [27]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)

    Listado_de_Sucursales=dfz[dfz['Día natural'] 
                              >datetime.date(2021, 6, 2)][
        'Oficina de Ventas Unifica'].unique()

    for y in Listado_de_Sucursales:
        #y='Huechuraba'
        print(y)
        dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)


        Listado_de_Materiales=dfy[dfy.Año==2021].Material.unique()

        for m in Listado_de_Materiales:
            #m=Listado_de_Materiales[0]
            material_time=time()
            #print(y,m)


            dfm=Df_ubic(dfy,
                        'Material',m).groupby('Día natural'
                                                 ).sum().reset_index()[['Día natural',y_col]]

            dfmPred=Predict_sem(dfm,8)[['ds', 'yhat_lower',
                                        'yhat_upper', 'yhat']].tail(8)
            
            dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
            dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
            dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
            dfmPred=dfmPred.reset_index()
            dfmPred.pop('index')
            df2=pd.concat([df2, dfmPred])
            print('El producto demoró',(time() - material_time),Z,y)
            print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')
    
print('tiempo bloque completo',(time() - start_time)/60, 'Min')


Santiago
Huechuraba


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.6324005126953125 Santiago Huechuraba
tiempo que lleva corriendo 0.0300841490427653 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.6709339618682861 Santiago Huechuraba
tiempo que lleva corriendo 0.0579413890838623 Min


...

El producto demoró 1.7389919757843018 Venta Directa VD FoodService
tiempo que lleva corriendo 9.714520112673442 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.5568361282348633 Venta Directa VD FoodService
tiempo que lleva corriendo 9.740492403507233 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 2.317488193511963 Venta Directa VD FoodService
tiempo que lleva corriendo 9.779125547409057 Min
El producto demoró 1.5713486671447754 Venta Directa VD FoodService
tiempo que lleva corriendo 9.805339713891348 Min
tiempo bloque completo 9.80534805059433 Min


In [28]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

In [43]:
df2.head()

,Día natural,Predicción Inferior,Predicción Superior,Predicción central,Material,Oficina de Ventas Unifica,Zona de ventas.1
0,2021-06-12,47.622361,1604.500692,856.754470,1020253,Huechuraba,Santiago
1,2021-06-13,-226.895459,1329.525243,582.661490,1020253,Huechuraba,Santiago
2,2021-06-14,-264.371925,1304.898453,502.017351,1020253,Huechuraba,Santiago
3,2021-06-15,-172.240421,1444.923201,658.550101,1020253,Huechuraba,Santiago
4,2021-06-16,20.990112,1564.555633,788.136734,1020253,Huechuraba,Santiago


### CSV SAVE 

In [44]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-02-05
2021-06-19


In [45]:
df2.to_csv('Data '+min_time+' Predicción CL ' +max_time+ 'Zona, Sucursal.csv', index=False)


### Exploracion del error 

In [48]:
Z

'Venta Directa'

In [49]:
y

'VD FoodService'

In [50]:
m

'1010048'

In [51]:
dfy['Material'].unique()

array(['1020253', '1010267', '1010078', '1010048'], dtype=object)

In [52]:
dfy[dfy.Año==2021]['Material'].unique()

array(['1020253', '1010267', '1010078', '1010048'], dtype=object)

In [53]:
dfy[dfy.Año==2021].groupby('Material').count()

,Sector,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
Material,,,,,,,,,,,,,,,,,
1010048,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
1010078,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1010267,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62
1020253,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20


In [54]:
dfy.groupby('Material').count()

,Sector,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año
Material,,,,,,,,,,,,,,,,,
1010048,270,270,270,270,270,270,270,270,270,270,270,270,270,270,270,270,270
1010078,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54
1010267,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410,410
1020253,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36


In [55]:
dfy_21=dfy[dfy.Año==2021]

In [56]:
dfy_21[dfy_21.Material=='1010695']

,Sector,Material,Tipo de cliente,Zona de ventas,Zona de ventas.1,Oficina de Ventas Unifica,Oficina de Ventas Unifica.1,Mes,Año natural/Semana,Día natural,Día de la semana,Kilos Venta KG,Venta Neta,Precio Promedio,Unidad Venta,Unidad Precio,Semana,Año


In [57]:
429.91*692.60

297755.666

## Iteración Zona

In [46]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)


    #Crear un listado de los materiales disponibles en la dat

    Listado_de_Materiales=dfz[dfz.Año==2021].Material.unique()     


    for m in Listado_de_Materiales:
        #m=Listado_de_Materiales[0]
        material_time=time()
        #print(y,m)


        dfm=Df_ubic(dfz,
                    'Material',m).groupby('Día natural'
                                             ).sum().reset_index()[['Día natural',y_col]]

        dfmPred=Predict_sem(dfm,8)[['ds', 'yhat_lower',
                                    'yhat_upper', 'yhat']].tail(8)
        #Material
        dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
        
        #Sucursal
        #dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]

        #Zona
        dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]

        dfmPred=dfmPred.reset_index()
        dfmPred.pop('index')
        df2=pd.concat([df2, dfmPred])
        print('El producto demoró',(time() - material_time),Z)
        print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')

print('tiempo bloque completo',(time() - start_time)/60, 'Min')


Santiago


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.342912197113037 Santiago
tiempo que lleva corriendo 0.07656582991282145 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 3.6187655925750732 Santiago
tiempo que lleva corriendo 0.13688693046569825 Min


...

El producto demoró 3.3914613723754883 Venta Directa
tiempo que lleva corriendo 4.196368618806203 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 2.8641700744628906 Venta Directa
tiempo que lleva corriendo 4.244113111495972 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 3.263408660888672 Venta Directa
tiempo que lleva corriendo 4.298519889513652 Min
El producto demoró 2.775852680206299 Venta Directa
tiempo que lleva corriendo 4.344784100850423 Min
tiempo bloque completo 4.344792425632477 Min


In [47]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

### CSV SAVE 

In [48]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-06-02
2021-06-19


In [49]:
df2.to_csv('Data'+min_time+' Predicción ' +max_time+ 'Zona.csv', index=False)


# Canal

## Iteración completa Zona, Sucursal

In [60]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)

    Listado_de_Sucursales=dfz[dfz['Día natural'] 
                              >datetime.date(2021, 6, 2)][
        'Oficina de Ventas Unifica'].unique()

    for y in Listado_de_Sucursales:
        #y='Huechuraba'
        print(y)
        dfy=Df_ubic(dfz,'Oficina de Ventas Unifica',y)
        
        
        Listado_de_Canales=dfy[dfy['Día natural'] 
                                  >datetime.date(2021, 6, 2)][
            'Tipo de cliente'].unique()
        
        for c in Listado_de_Canales:
            dfc=Df_ubic(dfy,'Tipo de cliente',c)

            #Crear un listado de los materiales disponibles en la data


            Listado_de_Materiales=dfc[dfc['Día natural'] 
                                  >datetime.date(2021, 6, 2)].Material.unique()     


            for m in Listado_de_Materiales:
                #m=Listado_de_Materiales[0]
                material_time=time()
                #print(y,m)


                dfm=Df_ubic(dfc,
                            'Material',m).groupby('Día natural'
                                                     ).sum().reset_index()[['Día natural',y_col]]
                if(dfm[
                    dfm['Día natural']>datetime.date(2021, 1, 2)
                        ].shape[0] >20):

                    dfmPred=Predict_sem(dfm,8)[['ds', 'yhat_lower',
                                                'yhat_upper', 'yhat']].tail(8)
                    #Material
                    dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
                    #Canal de ventas
                    dfmPred['Tipo de cliente']=[c for i in range(dfmPred.shape[0])]

                    #Sucursal
                    dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]

                    #Zona
                    dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
                    dfmPred=dfmPred.reset_index()
                    dfmPred.pop('index')
                    df2=pd.concat([df2, dfmPred])
                print('El producto demoró',(time() - material_time),Z,y)
                print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')

print('tiempo bloque completo',(time() - start_time)/60, 'Min')


Santiago


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Huechuraba


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 5.02265477180481 Santiago Huechuraba
tiempo que lleva corriendo 0.08861813545227051 Min


...

El producto demoró 2.643071413040161 Norte Copiapo
tiempo que lleva corriendo 57.00055242776871 Min
El producto demoró 2.6430816650390625 Norte Copiapo
tiempo que lleva corriendo 57.04461209376653 Min
El producto demoró 0.008987188339233398 Norte Copiapo
tiempo que lleva corriendo 57.04476188023885 Min
El producto demoró 0.01347208023071289 Norte Copiapo
tiempo que lleva corriendo 57.04499473174413 Min
Venta Directa


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


VD FoodService


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 4.612915754318237 Venta Directa VD FoodService
tiempo que lleva corriendo 57.12681746880214 Min
El producto demoró 4.429256916046143 Venta Directa VD FoodService
tiempo que lleva corriendo 57.20063841740291 Min
El producto demoró 0.009481191635131836 Venta Directa VD FoodService
tiempo que lleva corriendo 57.20081309080124 Min
tiempo bloque completo 57.20082142750422 Min


In [63]:
Z

'Centro Norte'

In [64]:
y

'Hijuelas'

In [65]:
c

'Industriales'

In [66]:
m

'1020253'

In [64]:
dfm

,Día natural,Kilos Venta KG
0,2019-03-04,220.0
1,2019-03-11,220.0
2,2019-03-12,221.0
3,2019-03-18,220.0
4,2019-03-21,220.0
5,2019-03-27,221.0
6,2019-04-01,220.0
7,2019-04-15,220.0
8,2019-04-22,220.0
9,2019-04-29,220.0


In [61]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

### CSV SAVE 

In [62]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-06-04
2021-06-19


In [63]:
df2.to_csv('Data'+min_time+' Predicción Por CANAL ' +max_time+ 'Zona, Sucursal.csv', index=False)


## Iteración Zona

In [ ]:
df2 = pd.DataFrame(columns=['ds', 'yhat_lower', 'yhat_upper', 'yhat', 'Material','Oficina de Ventas Unifica', 'Zona de ventas.1'])

Listado_de_zonas=data['Zona de ventas.1'].unique()

start_time = time()



for Z in Listado_de_zonas:
#Z='Santiago'
    print(Z)
    dfz=Df_ubic(data,'Zona de ventas.1',Z)

    Listado_de_Canales=dfz[dfz['Día natural'] 
                              >datetime.date(2021, 6, 2)][
        'Tipo de cliente'].unique()
    
    
    for c in Listado_de_Canales:
        dfc=Df_ubic(dfz,'Tipo de cliente',c)

        #Crear un listado de los materiales disponibles en la data
        Listado_de_Materiales=dfc[dfc.Año==2021].Material.unique()     


        for m in Listado_de_Materiales:
            #m=Listado_de_Materiales[0]
            material_time=time()
            #print(y,m)


            dfm=Df_ubic(dfc,
                        'Material',m).groupby('Día natural'
                                                 ).sum().reset_index()[['Día natural',y_col]]

            dfmPred=Predict_sem(dfm,8)[['ds', 'yhat_lower',
                                        'yhat_upper', 'yhat']].tail(8)
            #Material
            dfmPred['Material']=[m for i in range(dfmPred.shape[0])]
            #Canal de ventas
            dfmPred['Tipo de cliente']=[c for i in range(dfmPred.shape[0])]

            #Sucursal
            #dfmPred['Oficina de Ventas Unifica']=[y for i in range(dfmPred.shape[0])]
            
            #Zona
            dfmPred['Zona de ventas.1']=[Z for i in range(dfmPred.shape[0])]
            
            dfmPred=dfmPred.reset_index()
            dfmPred.pop('index')
            df2=pd.concat([df2, dfmPred])
            print('El producto demoró',(time() - material_time),Z,y)
            print('tiempo que lleva corriendo',(time() - start_time)/60, 'Min')

print('tiempo bloque completo',(time() - start_time)/60, 'Min')


Santiago


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.827568769454956 Santiago VD FoodService
tiempo que lleva corriendo 0.0332285205523173 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.7835307121276855 Santiago VD FoodService
tiempo que lleva corriendo 0.06297905445098877 Min


...

El producto demoró 1.6469132900238037 Centro Sur VD FoodService
tiempo que lleva corriendo 4.670866612593333 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.613884687423706 Centro Sur VD FoodService
tiempo que lleva corriendo 4.697789716720581 Min


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


El producto demoró 1.805548906326294 Centro Sur VD FoodService
tiempo que lleva corriendo 4.727898891766866 Min


### CSV SAVE 

In [57]:
df2.rename(columns = {'ds': 'Día natural', 'yhat': 'Predicción central',
                                'yhat_upper': 'Predicción Superior',
                                'yhat_lower': 'Predicción Inferior'}, inplace = True)

In [58]:
min_time=df2['Día natural'].min()
min_time=str(min_time).split(' ')[0]
print(min_time)
max_time=df2['Día natural'].max()
max_time=str(max_time).split(' ')[0]
print(max_time)

2021-02-14
2021-06-19


In [59]:
df2.to_csv('Data'+min_time+' Predicción Por CANAL ' +max_time+ 'Zona.csv', index=False)


# DEmanda

In [ ]:
#Nombre del producto
Fecha='08_06_21'

# =============================================================================
# Ajuste de los datos
# =============================================================================
File_name='Data 08_06_21 Predicción CL 2021-06-14'
col_names=pd.read_csv("Data/"+File_name+".csv",
                    encoding="utf-8",sep=";",nrows=0).columns
types_dict = {col: str for col in col_names}

Data = pd.read_csv("Data/"+File_name+".csv",
                   encoding="utf-8",sep=",",dtype=types_dict)
Data.head(1)

In [ ]:
Data.groupby('Material').sum()